<H1>Naive Bayes Sentiment Analysis Challenge</H1><br><br>
I really want to overfit this time. Then I'll go through the <br><br>
Sentiment raw data was taken from the <a href='https://archive.ics.uci.edu/ml/datasets/Sentiment+Labelled+Sentences'>UCI Machine Learning Repository.</a>

In [109]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import string
import re

In [110]:
data = pd.read_csv('amazon_cells_labelled.txt', engine='python', header=None, sep=None)
data.columns = ['text', 'sentiment']
data['positive_sentiment'] = np.where((data.sentiment == 1), True, False)

def get_words(series):
    words = []
    for item in series: 
        words += item #put all words in same list
    translator = str.maketrans('', '', string.punctuation)
    for i, word in enumerate(words):
        word = word.translate(translator)
        words[i] = word #strip away punctutation
    return words

positive_words = pd.Series(get_words(data[data['sentiment'] == 1].text.str.lower().str.split()))
negative_words = pd.Series(get_words(data[data['sentiment'] == 0].text.str.lower().str.split()))

In [111]:
neg = list(negative_words.value_counts().index)
pos = list(positive_words.value_counts().index)

In [112]:
keywords = neg + pos #simply grab all words for keywords

In [113]:
for key in keywords:
    re_string = '[^a-zA-Z]' + key + '[^a-zA-Z]' #to match words with spaces or punctuations around only
    data[str(key)] = data.text.apply(lambda x: bool(re.search(re_string, str(x), re.IGNORECASE)))

In [114]:
bnb = BernoulliNB()

In [115]:
overfit = bnb.fit(data[keywords], data.positive_sentiment)

In [116]:
correct_predictions = np.where((overfit.predict(data[keywords]) == data.positive_sentiment), True, False)
pd.Series(correct_predictions).value_counts()

True     767
False    233
dtype: int64

We still only have 77% accuracy on the training data...<br>
Let's try training on one half of the data set and running it against the other half.

In [117]:
holdout = BernoulliNB().fit(data.iloc[:500][keywords], data.positive_sentiment.iloc[:500])

In [118]:
correct_predictions = np.where((holdout.predict(data.iloc[500:][keywords]) == data.iloc[500:].positive_sentiment), True, False)
pd.Series(correct_predictions).value_counts()

True     258
False    242
dtype: int64

In [119]:
correct_predictions = np.where((holdout.predict(data.iloc[:500][keywords]) == data.iloc[:500].positive_sentiment), True, False)
pd.Series(correct_predictions).value_counts()

True     316
False    184
dtype: int64

Here we see about 64% accuracy with the training group. With the holdout group, we see about 50% accuracy.<br>
Let's compare these two models against the other data sets.

In [120]:
imdb_data = pd.read_csv('imdb_labelled.txt', engine='python', header=None, sep='\t', quoting=3)
imdb_data.columns = ['text', 'sentiment']
imdb_data['positive_sentiment'] = np.where((imdb_data.sentiment == 1), True, False)
for key in keywords:
    re_string = '[^a-zA-Z]' + key + '[^a-zA-Z]' #to match words with spaces or punctuations around only
    imdb_data[str(key)] = imdb_data.text.apply(lambda x: bool(re.search(re_string, str(x), re.IGNORECASE)))

In [121]:
correct_predictions = np.where((overfit.predict(imdb_data[keywords]) == imdb_data.positive_sentiment), True, False)
pd.Series(correct_predictions).value_counts()

True     610
False    390
dtype: int64

In [122]:
correct_predictions = np.where((holdout.predict(imdb_data[keywords]) == imdb_data.positive_sentiment), True, False)
pd.Series(correct_predictions).value_counts()

True     532
False    468
dtype: int64

In [123]:
yelp_data = pd.read_csv('yelp_labelled.txt', engine='python', header=None, sep='\t', quoting=3)
yelp_data.columns = ['text', 'sentiment']
yelp_data['positive_sentiment'] = np.where((yelp_data.sentiment == 1), True, False)
for key in keywords:
    re_string = '[^a-zA-Z]' + key + '[^a-zA-Z]' #to match words with spaces or punctuations around only
    yelp_data[str(key)] = yelp_data.text.apply(lambda x: bool(re.search(re_string, str(x), re.IGNORECASE)))

In [124]:
correct_predictions = np.where((overfit.predict(yelp_data[keywords]) == yelp_data.positive_sentiment), True, False)
pd.Series(correct_predictions).value_counts()

True     583
False    417
dtype: int64

In [125]:
correct_predictions = np.where((holdout.predict(yelp_data[keywords]) == yelp_data.positive_sentiment), True, False)
pd.Series(correct_predictions).value_counts()

True     515
False    485
dtype: int64